In [1]:
# Get the absolute path to the src directory.
# must do this to import modules from the src directory
import sys
import os
# sys.path.append(os.path.abspath("../src"))  # Linux
#sys.path.append(os.path.abspath("..\src"))  # Windows

# Get the absolute path to src
PROJECT_ROOT = os.path.abspath("..")  # Adjust based on your structure
SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"📌 Added {SRC_PATH} to sys.path")
    
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"📌 Added {PROJECT_ROOT} to sys.path")

📌 Added c:\Users\tan_poh_keam\OneDrive - Republic Polytechnic\1. Projects\UBTS\msg_app\src to sys.path
📌 Added c:\Users\tan_poh_keam\OneDrive - Republic Polytechnic\1. Projects\UBTS\msg_app to sys.path


In [4]:
# Standard Library Imports
import os
import base64
import pickle
import re 


# Third-Party Library Imports
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Applicaion library imports
from src.utils import sanitize_filename

### Utilities Functions

In [5]:
## Testing only


print(sanitize_filename("d\\/*?:'aaa<>|].ppt"))

TypeError: 'module' object is not callable

### Run #1 :Authehtication. Get the Gmail Services

For personal Gmail accounts, you must use OAuth 2.0 User Authentication . 
You cannot use Service account authentication. 
Service Accounts are designed for Google Workspace (formerly G Suite) organizations.

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

In [ ]:
### this is based on OA2. Installed on client. Need client permission.

## RUN 001

# Define the required Gmail API scopes

def authenticate_gmail(credentials="credentials.json", token="token.pickle"):
    creds = None
    #token_file = "token.pickle"
    

    # Load saved credentials (if available)
    if os.path.exists(token):
        with open(token, "rb") as token:
            creds = pickle.load(token)

    # If no valid credentials exist, start OAuth 2.0 flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials, SCOPES)
            creds = flow.run_local_server(port=0)  # Opens a browser for authentication

        # Save the credentials for next time
        with open(token, "wb") as token_file:
            pickle.dump(creds, token_file)

    # Initialize the Gmail API service
    service = build("gmail", "v1", credentials=creds)
    print("✅ Gmail API authentication successful!")
    return service


## Run #2: Save email as EML

In [ ]:
## This function is working.
## Email are saved as EML


def save_email_with_attachments_as_eml(service, msg_id, save_path="emails"):
    """
    Fetches an email by ID and saves it as an .eml file, including attachments.

    Args:
        service (googleapiclient.discovery.Resource): The authenticated Gmail API service object.
        msg_id (str): The Gmail message ID of the email to save.
        save_path (str, optional): Directory where the .eml file will be saved. Default is "emails".

    Returns:
        str: The file path of the saved .eml file.
    """
    try:
        # Fetch full email details in "raw" format to preserve attachments
        msg_detail = service.users().messages().get(userId="me", id=msg_id, format="raw").execute()

        # Decode the raw email content from Base64 (full RFC 822 email format)
        raw_email = base64.urlsafe_b64decode(msg_detail["raw"]).decode("utf-8")

        # Ensure the save directory exists
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        # Define file path (using message ID)
        eml_filename = f"email_{msg_id}.eml"
        eml_file_path = os.path.join(save_path, eml_filename)

        # Save as .eml file (which includes attachments)
        with open(eml_file_path, "w", encoding="utf-8") as eml_file:
            eml_file.write(raw_email)

        print(f"📩 Email (with attachments) saved as: {eml_file_path}")
        return eml_file_path

    except Exception as e:
        print(f"❌ Failed to save email as EML: {e}")
        return None

# Example usage
#email_id = "your-email-id"  # Replace with an actual email ID
#email_id = "1950408e07eaec6e"
#email_id = "195167f63a91f452"  # with 4x attachments
#save_email_with_attachments_as_eml(service, email_id)



### Run #3: Archive Emails and Attachments

In [ ]:
## This block of codes are working to save the email as html and download the attachments.



def extract_email_body(payload):
    """
    Recursively extracts the best available email body (HTML or plain text).
    """
    html_body, plain_text_body = None, None

    if "body" in payload and "data" in payload["body"]:
        content_type = payload.get("mimeType", "")

        if content_type == "text/html":
            return base64.urlsafe_b64decode(payload["body"]["data"]).decode("utf-8", errors="ignore")
        elif content_type == "text/plain":
            return f"<pre>{base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8', errors='ignore')}</pre>"

    if "parts" in payload:
        for part in payload["parts"]:
            content_type = part.get("mimeType", "")

            if content_type == "text/html":
                html_body = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8", errors="ignore")
            elif content_type == "text/plain":
                plain_text_body = f"<pre>{base64.urlsafe_b64decode(part['body']['data']).decode('utf-8', errors='ignore')}</pre>"

            if "parts" in part:
                nested_body = extract_email_body(part)
                if nested_body:
                    return nested_body

    return html_body or plain_text_body or "No content available."


def save_attachment(service, msg_id, part, save_path):
    """
    Saves an email attachment from a given message part.
    """
    try:
        if "filename" in part and part["filename"]:
            filename = sanitize_filename(part["filename"])
            attachment_id = part["body"].get("attachmentId")
            
            if attachment_id:
                attachment = service.users().messages().attachments().get(
                    userId="me", messageId=msg_id, id=attachment_id
                ).execute()

                data = base64.urlsafe_b64decode(attachment["data"])
                file_path = os.path.join(save_path, filename)

                with open(file_path, "wb") as f:
                    f.write(data)

                print(f"📎 Attachment saved: {file_path}")
                return file_path
    except HttpError as error:
        print(f"❌ Failed to download attachment: {error}")

    return None


def extract_email_and_attachments(service, msg_id, save_path="emails"):
    """
    Extracts an email, saves its headers and body as an HTML archive, and downloads all attachments.
    """
    try:
        msg_detail = service.users().messages().get(userId="me", id=msg_id, format="full").execute()
        headers = msg_detail["payload"]["headers"]

        subject, sender, recipient, date = "No Subject", "Unknown Sender", "Unknown Recipient", "Unknown Date"

        for header in headers:
            if header["name"] == "Subject":
                subject = header["value"]
            elif header["name"] == "From":
                sender = header["value"]
            elif header["name"] == "To":
                recipient = header["value"]
            elif header["name"] == "Date":
                date = header["value"]

        safe_subject = sanitize_filename(subject)
        email_folder = os.path.join(save_path, safe_subject)
        os.makedirs(email_folder, exist_ok=True)

        email_body = extract_email_body(msg_detail["payload"])

        # Save email as HTML
        email_html_content = f"""
        <html>
        <head>
            <meta charset="UTF-8">
            <style>
                body {{ font-family: Arial, sans-serif; padding: 20px; line-height: 1.6; }}
                .metadata {{ background: #f4f4f4; padding: 10px; border-radius: 5px; }}
                .metadata p {{ margin: 5px 0; }}
            </style>
        </head>
        <body>
            <div class="metadata">
                <p><strong>Subject:</strong> {subject}</p>
                <p><strong>From:</strong> {sender}</p>
                <p><strong>To:</strong> {recipient}</p>
                <p><strong>Date:</strong> {date}</p>
            </div>
            <hr>
            <div class="email-body">
                {email_body}
            </div>
        </body>
        </html>
        """

        email_html_path = os.path.join(email_folder, f"{safe_subject}.html")
        with open(email_html_path, "w", encoding="utf-8") as html_file:
            html_file.write(email_html_content)

        # Process attachments
        attachments = []
        if "parts" in msg_detail["payload"]:
            for part in msg_detail["payload"]["parts"]:
                attachment_path = save_attachment(service, msg_id, part, email_folder)
                if attachment_path:
                    attachments.append(attachment_path)

        print(f"📧 Email archived: {email_html_path}")
        return {"email_html": email_html_path, "attachments": attachments}

    except Exception as e:
        print(f"❌ Error extracting email: {e}")
        return None


# Example usage
# email_id = "your-email-id"  # Replace with a real Gmail message ID
# extract_email_and_attachments(service, email_id)


In [ ]:

email_id = "your-email-id"  # Replace with an actual email ID
email_id = "1950408e07eaec6e"
email_id = "195040dcd7df7aff"
#email_id = "195167f63a91f452"  # with 4x attachments


# Authenticate and initialize Gmail API

service = authenticate_gmail(
    credentials=r"..\secrets\credentials.json",
    token=r"..\secrets\token.pickle"
)

save_email_with_attachments_as_eml(service, email_id)

extract_email_and_attachments(service, email_id)

### The Tools for debugging

In [ ]:

def list_emails(service):

    results = service.users().messages().list(userId="me", maxResults=5).execute()
    messages = results.get("messages", [])

    if not messages:
        print("📭 No emails found.")
        return

    print("📩 Latest Emails:")
    for msg in messages:
        msg_detail = service.users().messages().get(userId="me", id=msg["id"]).execute()
        print(f"🆔 {msg['id']}, Subject: {msg_detail.get('snippet', 'No Subject')}")

# Run the function
list_emails(service)


In [ ]:
import base64

def list_emails_with_details(service, max_results=5):
    """
    Fetches and displays the latest emails with their real subject and full message content.

    Args:
        service (googleapiclient.discovery.Resource): The authenticated Gmail API service object.
        max_results (int, optional): The number of emails to retrieve. Default is 5.

    Returns:
        None: Prints the email details including ID, subject, and message content.
    """
    # Get the latest emails
    results = service.users().messages().list(userId="me", maxResults=max_results).execute()
    messages = results.get("messages", [])

    if not messages:
        print("📭 No emails found.")
        return

    print("📩 Latest Emails:")

    for msg in messages:
        # Fetch full email details
        msg_detail = service.users().messages().get(userId="me", id=msg["id"], format="full").execute()

        # Extract subject from headers
        headers = msg_detail["payload"]["headers"]
        subject = "No Subject"
        for header in headers:
            if header["name"] == "Subject":
                subject = header["value"]
                break

        # Extract message body (handling different formats)
        message_body = "No content"
        if "parts" in msg_detail["payload"]:
            for part in msg_detail["payload"]["parts"]:
                if part["mimeType"] == "text/plain":
                    message_body = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8")
                    break
        elif "body" in msg_detail["payload"] and "data" in msg_detail["payload"]["body"]:
            message_body = base64.urlsafe_b64decode(msg_detail["payload"]["body"]["data"]).decode("utf-8")

        # Print email details
        print(f"\n🆔 {msg['id']}")
        print(f"📌 Subject: {subject}")
        print(f"📜 Message: {message_body[:500]}...")  # Limiting output to 500 characters for readability

# Run the function
list_emails_with_details(service, max_results=100)
